In [ ]:
# 2023-05-23 - This code was created to quickly backup all of the good `loadedSessPickle.pkl` files from the session folders into the './output' subfolder in the session:
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-03_12-3-25/output/loadedSessPickle.pkl already in output folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-09_21-17-16/output/loadedSessPickle.pkl already in output folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet11-04_21-20-3/output/loadedSessPickle.pkl already in output folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-03_21-26-8/output/loadedSessPickle.pkl already in output folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-09_22-4-5/output/loadedSessPickle.pkl already in output folder.
# ...

In [38]:
%config IPCompleter.use_jedi = False
%load_ext autoreload
%autoreload 3
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
from copy import deepcopy
from datetime import datetime, timedelta
from typing import Optional, Dict, List


## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file, FileList
from pyphocorehelpers.Filesystem.path_helpers import convert_filelist_to_new_parent
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_files_metadata

# import glob # for finding .whl file after building binary repo
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun

global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/home/halechr/FastData'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

completed_pipeline_filename = 'loadedSessPickle.pkl'
completed_global_computations_filename = 'global_computation_results.pkl'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Old (Pre 2023-09-21):

In [ ]:
# Use glob to find the first generated .whl file in the dist/ directory
found_whl_files = glob.glob('*.whl')
found_whl_files = [a for a in found_whl_files if not a.endswith('current.whl')] # exclude the symlink from the search

In [ ]:
external_computed_ripple_filepath = session.basepath.joinpath('ripple_df.pkl')
## try the '.ripple.npy' ripples:
active_file_suffix = '.ripple.npy'
external_computed_ripple_filepath = fp.with_suffix(active_file_suffix)



In [ ]:
## Moves all previously saved loadedSessPickle.pkl files to the output folder.
correctly_placed_output_files = []
search_path: Path = Path("/home/halechr/turbo/Data/KDIBA/").resolve()
assert search_path.exists()
for p in search_path.rglob(completed_pipeline_filename):
    if p.parent.name != 'output':
        # If not already in the 'output/' subfolder, move it there.
        print(f'p: {p}')
        curr_output_dir = p.parent.joinpath('output')
        curr_output_dir.mkdir(exist_ok=True)
        # print(f'curr_output_dir: {curr_output_dir}')
        new_path = p.replace(curr_output_dir.joinpath(p.name))
        print(f'\t new_path: {new_path}')
        correctly_placed_output_files.append(p)
    else:
        print(f'p: {p} already in output folder.')
        correctly_placed_output_files.append(p)
    
correctly_placed_output_files

In [ ]:
completed_pipeline_filename

In [3]:

## Build Pickle Path:
pkl_path = 'global_batch_result_2023-06-08.pkl'
csv_path = 'global_batch_result_2023-06-08.csv'
h5_path = 'global_batch_result_2023-06-08.h5'

global_batch_result_file_path = Path(global_data_root_parent_path).joinpath(h5_path).resolve() # Use Default

batch_progress_df = BatchRun.load_batch_progress_df_from_h5(global_batch_result_file_path)
batch_progress_df

updated_batch_progress_df = BatchRun.rebuild_basedirs(batch_progress_df, global_data_root_parent_path)
updated_batch_progress_df

updated_good_only_batch_progress_df = updated_batch_progress_df[updated_batch_progress_df['locally_is_ready']].copy()
updated_good_only_batch_progress_df

In [ ]:
# Generate code that can be pasted into the current "ReviewOfWork-*.ipynb" notebook to load the good sessions:
print(",\n".join([a_ctxt.get_initialization_code_string() for a_ctxt in updated_good_only_batch_progress_df['context'].to_list()]))


# 2023-09-21 - Mirror Slow Data files to much faster SSD: , Path(r'/home/halechr/FastData')

In [28]:
from typing import Optional, Dict, List
from datetime import datetime, timedelta
from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file


In [22]:

source_data_root = Path(r'/media/MAX/Data')
dest_data_root = Path(r'/home/halechr/FastData')
assert source_data_root.exists(), f"source_data_root: {source_data_root} does not exist! Is the right computer's config commented out above?"
assert dest_data_root.exists(), f"dest_data_root: {dest_data_root} does not exist! Is the right computer's config commented out above?"

oldest_modified_date = (datetime.now() - timedelta(days=5))

In [9]:
# Find the files and build the movedicts:
found_session_pickle_files = discover_data_files(source_data_root, glob_pattern='loadedSessPickle.pkl', recursive=True)
found_global_computation_results_files = discover_data_files(source_data_root, glob_pattern=f'output/{completed_global_computations_filename}', recursive=True)
file_movedict_session_pickle_files = generate_copydict(source_data_root, dest_data_root, found_files=found_session_pickle_files, only_files_newer_than=oldest_modified_date)
file_movedict_global_computation_results_pickle_files = generate_copydict(source_data_root, dest_data_root, found_files=found_global_computation_results_files, only_files_newer_than=oldest_modified_date)

[PosixPath('/media/MAX/Data/Bapun/RatN/Day4OpenField/loadedSessPickle.pkl'),
 PosixPath('/media/MAX/Data/Bapun/RatS/Day5TwoNovel/loadedSessPickle.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-26-53/loadedSessPickle.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22-43/loadedSessPickle.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-55-31/loadedSessPickle.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/gor01/one/2006-6-13_14-42-6/loadedSessPickle.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-40-19/loadedSessPickle.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/gor01/two/2006-6-08_15-46-47/loadedSessPickle.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-16-25/loadedSessPickle.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/gor01/two/2006-6-09_22-24-40/loadedSessPickle.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/gor01/two/2006-6-12_16-53-46/loa

### Actually perform copy operations. This will take a while

In [37]:
moved_files_dict_session_pickle_files = copy_movedict(file_movedict_session_pickle_files)
moved_files_dict_session_pickle_files

# moved_files_dict = copy_files(filelist_source, filelist_dest)

moving "/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results.pkl"
		 -> "/home/halechr/FastData/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results.pkl"...
done.
moving "/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/global_computation_results.pkl"
		 -> "/home/halechr/FastData/KDIBA/gor01/one/2006-6-09_1-22-43/output/global_computation_results.pkl"...
done.
moving "/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-40-19/output/global_computation_results.pkl"
		 -> "/home/halechr/FastData/KDIBA/gor01/two/2006-6-07_16-40-19/output/global_computation_results.pkl"...
done.
done copying 3 of 3 files.


{PosixPath('/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results.pkl'): PosixPath('/home/halechr/FastData/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/global_computation_results.pkl'): PosixPath('/home/halechr/FastData/KDIBA/gor01/one/2006-6-09_1-22-43/output/global_computation_results.pkl'),
 PosixPath('/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-40-19/output/global_computation_results.pkl'): PosixPath('/home/halechr/FastData/KDIBA/gor01/two/2006-6-07_16-40-19/output/global_computation_results.pkl')}

In [ ]:
moved_files_dict_global_computation_results_pickle_files = copy_movedict(file_movedict_global_computation_results_pickle_files)
moved_files_dict_global_computation_results_pickle_files

# 2023-11-20 - Discovery of folder modification times from matching files

In [ ]:
Write a python function that


In [ ]:

def determine_directory_modification_time(target_